In [76]:
import logging
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

In [77]:
# Reading the FITS file
image = fits.open("D:\\Files\\Background Magnitude Group2\\stars.fits")[0].data

In [78]:
# Calculating flux of rectangles
UL_point1i = 673
UL_point1j = 823
DR_point1i = 847
DR_point1j = 1222
UL_point2i = 1798
UL_point2j = 1207
DR_point2i = 1952
DR_point2j = 1445

R_flux = 0
pixel_counter = 0

# Calculate flux for the first rectangle
for i in range(UL_point1i, DR_point1i + 1):
    for j in range(UL_point1j, DR_point1j + 1):
        R_flux += image[i][j]
        pixel_counter += 1

# Calculate flux for the second rectangle
for i in range(UL_point2i, DR_point2i + 1):
    for j in range(UL_point2j, DR_point2j + 1):
        R_flux += image[i][j]
        pixel_counter += 1


In [79]:
# Finding the center of the star
def get_croped_image(image, center_of_mass):
    i_center_of_mass = center_of_mass[0]
    j_center_of_mass = center_of_mass[1]
    croped_image = np.zeros((31, 31), dtype=int)
    i_counter = 0
    for i in range(i_center_of_mass - 15, i_center_of_mass + 16):
        j_counter = 0
        for j in range(j_center_of_mass - 15, j_center_of_mass + 16):
            croped_image[i_counter][j_counter] = image[i][j]
            j_counter += 1
        i_counter += 1
    return croped_image

def get_center_of_mass(image, i_of_star, j_of_star):
    """
    Calculate the center of mass of a star.

    :param image: ndarray containing the star image
    :param i_of_star: row index of the star
    :param j_of_star: column index of the star
    :return: 2D array representing the position of the center of mass
    """
    total_mass = 0
    weight_vector = np.array([0., 0.])
    for i in range(i_of_star - 20, i_of_star + 21):
        for j in range(j_of_star - 20, j_of_star + 21):
            weight_vector += image[i][j] * np.array([i, j])
            total_mass += image[i][j]
    center_of_mass = (np.round(weight_vector / total_mass)).astype(int)
    return center_of_mass, get_croped_image(image, center_of_mass)

In [83]:
def calculate_signal(image, i_of_star, j_of_star, radius):
    """
    Calculate the signal within a given radius around the star.

    :param image: ndarray containing the star image
    :param i_of_star: row index of the star
    :param j_of_star: column index of the star
    :param radius: radius around the star
    :return: signal value
    """
    signal = 0
    pixel_counter = 0
    for i in range(i_of_star - radius, i_of_star + radius + 1):
        for j in range(j_of_star - radius, j_of_star + radius + 1):
            x_val = i - i_of_star
            y_val = j - j_of_star
            if (x_val ** 2 + y_val ** 2 <= radius ** 2):
                signal += image[i][j]
                pixel_counter += 1
    return signal

In [86]:
# Finding the Full Width at Half Maximum (FWHM) of the star
def get_FWHM(data):
    """
    Calculate the Full Width at Half Maximum (FWHM) of a star.

    :param data: 2D array representing the star image
    :return: FWHM value
    """
    max_data = np.max(data)
    half_max_data = (max_data / 2).astype(int)
    indices = np.where(data >= half_max_data)
    x_indices = indices[0]
    y_indices = indices[1]
    FWHM_x = (x_indices.max() - x_indices.min())
    FWHM_y = (y_indices.max() - y_indices.min())
    FWHM = (FWHM_x + FWHM_y) / 2
    return int(FWHM)

# Get the center of mass and cropped image of the star
center_of_mass, croped_image = get_center_of_mass(image, 790, 426)

# Calculate the radius of the star using its FWHM
radius = np.round(get_FWHM(croped_image)).astype(int)

# Calculate the flux of the star
F_star = calculate_signal(image, center_of_mass[0], center_of_mass[1], radius)


In [88]:
# Calculating magnitude
m_star = 8.88
magnitude_of_rectangles = -2.5 * np.log10(R_flux / F_star) + m_star
flux_per_arc_sec2 = R_flux / (pixel_counter * (0.89 ** 2))
magnitude_per_arc_sec2 = -2.5 * np.log10(flux_per_arc_sec2 / F_star) + m_star

# Printing results
print("Rectangles flux:", R_flux, "\n"
      "Number of pixels in rectangles:", pixel_counter, "\n"
      "Pixel size: 0.89\"", "\n"
      "Star flux:", F_star, "\n"
      "Magnitude of rectangles:", magnitude_of_rectangles, "\n"
      "Flux per arc sec^2:", flux_per_arc_sec2, "\n"
      "Magnitude per arc sec^2:", magnitude_per_arc_sec2, "\n")

rectangles flux:  374383660 
number of pixels in rectangles:  107045 
pixel size: 0.89" 
star flux:  28872537 
magnitude of rectangles:  -2.3763237182733654 
flux per arc sec^2:  4415.404397843908 
magnitude per arc sec^2:  18.418786151719328 

